### Importing Libraries

In [1]:
from parcels import FieldSet, Field, ParticleSet, JITParticle, AdvectionRK4
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

from datetime import timedelta

### Verifying the structure and dimentions of the data files

In [2]:
# Open the SSC dataset
ssc_ds = xr.open_dataset("Data/Processed_SSC_Data.nc")

# Print the SSC dataset
print("SSC Dataset:\n")
print(ssc_ds)
print("=" * 125)

# Open the wind dataset
wind_ds = xr.open_dataset("Data/Processed_Wind_Data.nc")

# Print the wind dataset
print("\nWind Dataset:\n")
print(wind_ds)
print("=" * 125)

# Close datasets after inspection
ssc_ds.close()
wind_ds.close()

SSC Dataset:

<xarray.Dataset>
Dimensions:  (lat: 25, lon: 22, time: 2091)
Coordinates:
  * lat      (lat) float32 35.74 35.77 35.79 35.81 ... 36.21 36.23 36.26 36.28
  * lon      (lon) float32 13.92 13.96 14.0 14.04 ... 14.65 14.69 14.73 14.77
  * time     (time) datetime64[ns] 2021-01-01 2021-01-01T12:00:00 ... 2023-11-12
Data variables:
    u        (time, lat, lon) float64 ...
    v        (time, lat, lon) float64 ...
    stdu     (time, lat, lon) float64 ...
    stdv     (time, lat, lon) float64 ...
    cov      (time, lat, lon) float64 ...
    velo     (time, lat, lon) float64 ...
    head     (time, lat, lon) float64 ...
Attributes: (12/17)
    NC_GLOBAL.Title:                   Near-Real time Surface Ocean Velocity
    NC_GLOBAL.origin:                  BARK (measured);POZZ (measured);
    NC_GLOBAL.source:                  HF Radar Derived Surface Currents obta...
    NC_GLOBAL.history:                 08-Jun-2023 14:46:23
    NC_GLOBAL.grid_type:               REGULAR
    NC_

### Creating a FieldSet

In [3]:
fieldset = FieldSet.from_netcdf(
    filenames={
        "U": "Data/Processed_SSC_Data.nc",
        "V": "Data/Processed_SSC_Data.nc",
        "U_wind": "Data/Processed_Wind_Data.nc",
        "V_wind": "Data/Processed_Wind_Data.nc"
    },
    variables={
        "U": "u",
        "V": "v",
        "U_wind": "u10",
        "V_wind": "v10"
    },
    dimensions={
        "U": {"lon": "lon", "lat": "lat", "time": "time"},
        "V": {"lon": "lon", "lat": "lat", "time": "time"},
        "U_wind": {"lon": "longitude", "lat": "latitude", "time": "time"},
        "V_wind": {"lon": "longitude", "lat": "latitude", "time": "time"}
    },
    allow_time_extrapolation=True
)

### Initializing the Particles

In [4]:
# Example coordinates for particle release (adjust according to your study)
# Let's say we want to release particles from two locations
lat_release = [35.9, 35.95]  # Example latitudes
lon_release = [14.5, 14.55]  # Example longitudes

# Create a ParticleSet with JITParticle
pset = ParticleSet(fieldset=fieldset,
                   pclass=JITParticle,  # JITParticle allows for fast execution
                   lon=lon_release,
                   lat=lat_release)

### Define Simulation Parameters

In [6]:
# Define the simulation runtime and timestep
runtime = timedelta(days=250)  # Simulate for 250 days
total_minutes = runtime.total_seconds() / 60  # Convert runtime to minutes

# Decide on your desired timestep, for example, 10 minutes
dt_new = 10  # Change this to your desired timestep in minutes

# Calculate the number of timesteps required with the new timestep
num_timesteps = total_minutes / dt_new

# Calculate the actual timestep duration as a timedelta
dt = timedelta(minutes=dt_new)

# Define the simulation runtime and timestep
#runtime = timedelta(days=10)  # Simulate for 10 days
#dt = timedelta(minutes=5)  # Integration timestep of 5 minutes

### Execute the Simulation

In [ ]:
# Define recovery kernel for deleting particles that go out of bounds
def DeleteParticle(particle, fieldset, time):
    particle.delete()

# Execute the simulation
output_file = pset.ParticleFile(name="debris_simulation_output.zarr", outputdt=timedelta(hours=1))  # Save output every hour
pset.execute(
    AdvectionRK4,  # Advection kernel 
    runtime=runtime,
    dt=dt,
    output_file=output_file
)

INFO: Output files are stored in debris_simulation_output.zarr.
 34%|███▍      | 7312200.0/21600000.0 [04:51<18:24, 12936.24it/s]

### Visualise

In [ ]:
# Load the simulation output
ds = xr.open_zarr("debris_simulation_output.zarr")

# Plot trajectories
plt.figure(figsize=(10, 6))
plt.scatter(ds.lon, ds.lat, s=1, c=ds.time, cmap='viridis', marker='o')
plt.colorbar(label='Time')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('Particle Trajectories')
plt.show()

In [ ]:
import numpy as np
import xarray as xr
from matplotlib import pyplot as plt, animation

# Load the simulation output
ds = xr.open_zarr("debris_simulation_output.zarr")

fig, ax = plt.subplots(figsize=(10, 6))
scat = ax.scatter([], [], s=1, cmap='viridis', marker='o')

# Set the axes limits and labels
ax.set_xlim(ds.lon.min(), ds.lon.max())
ax.set_ylim(ds.lat.min(), ds.lat.max())
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')

# Initialize the colors based on the first time step
times = ds.time.values.astype('float')  # Convert times to float for coloring
scat.set_array((times - times.min()) / (times.max() - times.min()))  # Normalize color

def init():
    scat.set_offsets(np.empty((0, 2)))
    return scat,

def animate(i):
    time_id = np.where(ds.time == ds.time[i])
    positions = np.c_[ds.lon[time_id].values.flatten(), ds.lat[time_id].values.flatten()]
    scat.set_offsets(positions)
    return scat,

# Create the animation
anim = animation.FuncAnimation(fig, animate, init_func=init, frames=len(ds.time), interval=100, blit=True)

# Display the animation
plt.show()

# To save the animation, use the following line
# anim.save('particle_trajectories.mp4', writer